In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
from scipy import io as sio
#from keras.utils.np_utils import to_categorical

In [5]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers

##### This is where the issue is!
##### The "num_classes" must be 26. NOT 37.
##### Although 26 gives an error I cant make sense of.

In [13]:
#num_classes = 26    THIS IS THE ISSUE!!!!
num_classes = 26
batch_size = 1000
epochs = 20
neurons = 800

mat_contents = sio.loadmat('ANN-Handwritten-Char-Recognition-master/matlab/emnist-letters.mat')
data = mat_contents['dataset']

X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

val_start = X_train.shape[0] - X_test.shape[0]
X_val = X_train[val_start:X_train.shape[0],:]
y_val = y_train[val_start:X_train.shape[0]]
X_train = X_train[0:val_start,:]
y_train = y_train[0:val_start]

#X_train = X_train.reshape( (X_train.shape[0], 28, 28), order='F')

y_train -=1
y_val -=1
y_test -=1

#convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], "training samples")
print(X_val.shape[0], "validation samples")
print(X_test.shape[0], "test samples")
print(X_val.shape, y_val.shape)

104000 training samples
20800 validation samples
20800 test samples
(20800, 784) (20800, 26)


In [14]:
model = tf.keras.Sequential()
model.add(layers.Dense(neurons, activation='relu', input_shape=(784,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(neurons, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 800)               628000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 800)               640800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 26)                20826     
Total params: 1,289,626
Trainable params: 1,289,626
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_val, y_val))

Train on 104000 samples, validate on 20800 samples
Epoch 1/20
104000/104000 [==============================] - 4s 36us/sample - loss: 17.8366 - accuracy: 0.5146 - val_loss: 1.3375 - val_accuracy: 0.6907
Epoch 2/20
104000/104000 [==============================] - 4s 36us/sample - loss: 1.3612 - accuracy: 0.6827 - val_loss: 0.8141 - val_accuracy: 0.7700
Epoch 3/20
104000/104000 [==============================] - 4s 38us/sample - loss: 0.8766 - accuracy: 0.7677 - val_loss: 0.5781 - val_accuracy: 0.8312
Epoch 4/20
104000/104000 [==============================] - 5s 48us/sample - loss: 0.6905 - accuracy: 0.8052 - val_loss: 0.5067 - val_accuracy: 0.8488
Epoch 5/20
104000/104000 [==============================] - 4s 40us/sample - loss: 0.6020 - accuracy: 0.8264 - val_loss: 0.4697 - val_accuracy: 0.8616
Epoch 6/20
104000/104000 [==============================] - 4s 38us/sample - loss: 0.5369 - accuracy: 0.8416 - val_loss: 0.4488 - val_accuracy: 0.8698
Epoch 7/20
104000/104000 [================

In [16]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.47535962739865334
Test accuracy: 0.8960096


# ignore for now

### num_classes = 28
### first Layer = 512, second layer = 512
#####             5  epochs, 128  batch size - 61.13%
#####             5  epochs,  64  batch size - 60.20%
#####             5  epochs, 128  batch size - 80.91%
#####             10 epochs, 128  batch size - 66.65%
#####             15 epochs, 128  batch size - 63.86%
#####             7  epochs, 128  batch size - 78.78%
#####             7  epochs, 256  batch size - 83.56%
#####             15 epochs, 256  batch size - 82.96%
#####             10 epochs, 256  batch size - 84.33%
#####             15 epochs, 512  batch size - 86.56%
#####             20 epochs, 512  batch size - 87.17%
#####             20 epochs, 1024 batch size - 88.74%
#####             50 epochs, 1024 batch size - 89.59%
#####             50 epochs, 2048 batch size - 89.97%
#####             100epochs, 4098 batch size - 90.44%
            
### num_classes = 37
### first Layer = 512, second layer = 512
#####             50 epochs, 4098 batch size - 90.28%
#####             50 epochs, 2048 batch size - 90.48%
#####             100epochs, 2048 batch size - 90.50%
#####             100epochs, 1024 batch size - 89.50%                        

### first Layer = 1024, second layer = 1024
#####             50 epochs, 1024 batch size - 89.14%
#####             50 epochs, 2048 batch size - 91.14%
#####             50 epochs, 4096 batch size - 90.39%
            
### first Layer = 2048, second layer = 2048
#####             50 epochs, 1024 batch size - 90.06%
#####             50 epochs, 2048 batch size - 90.24%
#####             50 epochs, 4096 batch size - 90.78%
            
            